In [1]:
import os
import cv2
import numpy as np

# 이미지 폴더 및 마스크 폴더 경로
image_dir = '/home/knuvi/Desktop/song/cucumber-image/data/oi_seg/images'
mask_dir = '/home/knuvi/Desktop/song/cucumber-image/data/oi_seg/masks'
output_dir = '/home/knuvi/Desktop/song/cucumber-image/data/oi_seg/cropped_objects'

# 알파 채널 적용하여 이미지를 저장하는 함수
def save_cropped_object(image_path, mask_path, output_dir):
    # 이미지와 마스크 파일 이름
    image_name = os.path.basename(image_path)
    mask_name = os.path.basename(mask_path)
    
    # 이미지와 마스크 로드
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if image is None or mask is None:
        print(f"이미지 또는 마스크 로드 실패: {image_name}, {mask_name}")
        return

    # 마스크가 활성화된 (255인) 영역만 선택
    object_mask = mask == 255

    # 알파 채널 추가 (이미지 크기에 맞는 투명 채널 생성)
    bgr_with_alpha = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
    
    # 배경을 투명하게 처리
    bgr_with_alpha[:, :, 3] = np.where(object_mask, 255, 0)

    # 활성화된 마스크 영역만 추출하여 객체 크롭
    x, y, w, h = cv2.boundingRect(object_mask.astype(np.uint8))
    cropped_object = bgr_with_alpha[y:y+h, x:x+w]

    # 파일 저장 경로 설정 (PNG 포맷으로 저장)
    output_filename = f"cropped_{os.path.splitext(mask_name)[0]}.png"
    output_path = os.path.join(output_dir, output_filename)

    # 디렉토리 확인 및 생성
    os.makedirs(output_dir, exist_ok=True)

    # 크롭된 객체 이미지 저장
    cv2.imwrite(output_path, cropped_object)
    print(f"객체 저장됨: {output_path}")

# 이미지와 마스크 처리 함수
def process_images_and_masks(image_dir, mask_dir, output_dir):
    # 이미지 파일과 동일한 마스크 파일 찾기
    for image_file in os.listdir(image_dir):
        image_path = os.path.join(image_dir, image_file)
        
        # 동일한 이름을 가진 마스크 파일 찾기
        mask_file_prefix = os.path.splitext(image_file)[0]
        matching_masks = [f for f in os.listdir(mask_dir) if f.startswith(mask_file_prefix)]

        for mask_file in matching_masks:
            mask_path = os.path.join(mask_dir, mask_file)
            save_cropped_object(image_path, mask_path, output_dir)
        break


In [2]:

# 실행
process_images_and_masks(image_dir, mask_dir, output_dir)

객체 저장됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/cropped_objects/cropped_oi_140_3_oi_140_3_0_00.png
객체 저장됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/cropped_objects/cropped_oi_140_3_oi_140_3_1_00.png
